In [1]:
# imports
from tueplots import bundles
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from tueplots  import figsizes

import sys

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '.')

In [15]:
from analysis import sweep2df, format_violin, RED, BLUE
from care_nl_ica.models.sinkhorn import learn_permutation

In [3]:
plt.rcParams.update(bundles.neurips2022(usetex=True))
plt.rcParams.update({
    'text.latex.preamble': [r'\usepackage{amsfonts}', # mathbb
                            r'\usepackage{amsmath}'] # boldsymbol
})

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/_collections_abc.py:834: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [72]:
# Constants
ENTITY = "causal-representation-learning"
PROJECT = "nl-causal-representations"

# W&B API
api = wandb.Api(timeout=200)
runs = api.runs(ENTITY + "/" + PROJECT)

# Data loading

In [78]:
true_unmix_jacobians_monti

[array({'mean': 2.8133716583251953, '_type': 'torch.Tensor', '10%': -8.083252286911009, '25%': 0, 'var': 272.113037109375, 'size': 64, '75%': 1.6542763710021973, '90%': 22.810142898559583, 'min': -36.085296630859375, 'max': 68.98664855957031},
       dtype=object),
 array([[  6.13943481,   0.        ,   0.        ,   0.        ,
           0.        ],
        [ -1.69066799,   1.39582276,   0.        ,   0.        ,
           0.        ],
        [  6.37966299,   0.31269157,  -2.2214396 ,   0.        ,
           0.        ],
        [-31.32329559,   3.20180988,   4.47699261,  -4.41901827,
           0.        ],
        [ 98.1496048 ,  -6.92886639, -12.47439098,  11.51362228,
           3.92419839]]),
 array([[-4.73548079,  0.        ,  0.        ,  0.        ,  0.        ],
        [-3.3953855 , -6.37604856,  0.        ,  0.        ,  0.        ],
        [16.31355476, 13.23537922, -3.04217029,  0.        ,  0.        ],
        [22.22460365, 15.32792759, -3.70758939,  1.73813486,  

In [33]:

SWEEP_ID = "vfv1je0d"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
sweep.runs[59].logged_artifacts()[0].get("dep_mat_table").get_column("dep_mat", "numpy")#.use_artifact("dep_mat_table")
# artifact = run.use_artifact('causal-representation-learning/experiment/run-iczvn8od-dep_mat_table:v0', type='run_table')
# artifact.get("dep_mat_table")

array([[ 1.93882871,  1.40007854,  1.50939941, -1.52444494,  0.01124954,
         0.00467368,  1.57117176,  1.79014182, -0.01026418]])

In [11]:
SWEEP_ID = "gz1yqsuk"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"sem_sweep_{SWEEP_ID}"
print(f"Loading sweep with {SWEEP_ID=}")
df1, (true_unmix_jacobians1, est_unmix_jacobians1, permute_indices1) = sweep2df(sweep.runs, filename, save=True, load=True)

SWEEP_ID = "5gzpzb23"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"sem_sweep_{SWEEP_ID}"
print(f"Loading sweep with {SWEEP_ID=}")
df2, (true_unmix_jacobians2, est_unmix_jacobians2, permute_indices2) = sweep2df(sweep.runs, filename, save=True, load=True)

df = df1.append(df2)
true_unmix_jacobians = true_unmix_jacobians1 +true_unmix_jacobians2
est_unmix_jacobians = est_unmix_jacobians1+est_unmix_jacobians2
permute_indices = permute_indices1+permute_indices2

Loading sweep with SWEEP_ID='gz1yqsuk'
Loading sweep with SWEEP_ID='5gzpzb23'


## 3D SEM

In [99]:

SWEEP_ID = "vfv1je0d" #"nz4r5d8a"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"sem_3d_sweep_{SWEEP_ID}"
df3d, (true_unmix_jacobians3d, est_unmix_jacobians3d, permute_indices3d) = sweep2df(sweep.runs, filename, save=True, load=False)

## 5D SEM

In [86]:
SWEEP_ID = "3ldi48id"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"sem_3d_sweep_{SWEEP_ID}"
df5d, (true_unmix_jacobians5d, est_unmix_jacobians5d, permute_indices5d) = sweep2df(sweep.runs, filename, save=True, load=False)

## 8D SEM

## 10D SEM

## MLP from Monti et al.

In [80]:
est_unmix_jacobians_monti

[array({'mean': 0.17256055772304535, 'min': -23.18301010131836, '_type': 'torch.Tensor', '25%': -0.37722308188676834, 'max': 12.382342338562012, 'var': 19.386812210083008, '10%': -2.902764678001403, '90%': 3.052418327331543, 'size': 64, '75%': 1.0104133635759354},
       dtype=object),
 array([[-6.15742493e+00, -2.75305100e-03,  7.83871859e-03,
         -4.32511047e-03,  6.34156517e-04],
        [ 3.04651318e+01, -3.10626674e+00, -4.36681366e+00,
          4.31507874e+00, -9.31978691e-03],
        [ 1.80417955e+00, -1.40463102e+00, -1.00446185e-02,
          2.04220153e-02,  5.05587878e-03],
        [ 9.34721069e+01, -6.57836294e+00, -1.19044800e+01,
          1.09991074e+01,  3.74103117e+00],
        [ 6.38666296e+00,  3.17610204e-01, -2.21809959e+00,
          9.24070308e-04,  3.59703181e-03]]),
 array([[ 2.12413521e+01,  1.47355518e+01, -3.51345396e+00,
          1.67496777e+00,  1.62693392e-02],
        [ 1.60499973e+01,  1.30976744e+01, -2.96611786e+00,
         -4.00109682e-04,  

In [81]:
SWEEP_ID = "fhaza97x"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"monti_sweep_{SWEEP_ID}"
df_monti, (true_unmix_jacobians_monti, est_unmix_jacobians_monti, permute_indices_monti) = sweep2df(sweep.runs, filename, save=True, load=False)

8
5
5
5
5
3
3
10
8
5
3
3
3


/Users/patrik.reizinger/Documents/GitHub/nl-causal-representations/venv/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [76]:
df_monti

,name,dim,permute,variant,n_mixing_layer,use_sem,nonlin_sem,force_chain,force_uniform,mcc,val_loss
0,distinctive-sweep-13,8,False,3,1,False,False,False,False,0.879461,0.488243
1,dulcet-sweep-12,5,False,3,1,False,False,False,False,0.999741,1.940162
2,decent-sweep-11,5,False,3,1,False,False,False,False,0.999712,1.923173
3,upbeat-sweep-10,5,False,3,1,False,False,False,False,0.805662,3.030349
4,hearty-sweep-9,5,False,3,1,False,False,False,False,0.814941,3.105927
5,comfy-sweep-7,3,False,3,1,False,False,False,False,0.999954,4.486361
6,eager-sweep-8,3,False,3,1,False,False,False,False,0.999953,4.434566
7,desert-sweep-20,10,False,3,1,False,False,False,False,0.825161,0.193559
8,misty-sweep-15,8,False,3,1,False,False,False,False,0.899726,0.484964
9,feasible-sweep-10,5,False,3,1,False,False,False,False,0.999930,1.939643


# Pre-processing

In [126]:
def learning_stats(df, true_unmix_jacobians, est_unmix_jacobians, permute_indices, hamming_threshold=1e-2):
    for dim in df.dim.unique():
        for nl in [False, True]:
            success = []
            hamming = []
            for (nl_sem, j_gt, j_est, permute) in zip(df.nonlin_sem, true_unmix_jacobians, est_unmix_jacobians, permute_indices):
                if j_gt.shape[0] == dim and nl_sem is nl:
                    s, h = learn_permutation(j_gt, j_est, permute, triu_weigth=20., tril_weight=10.,diag_weight=6., num_steps=5000, lr=1e-4, verbose=True,drop_smallest=True, threshold=None, binary=True, hamming_threshold=hamming_threshold)

                    success.append(s)
                    hamming.append(h)

            mcc = df.mcc[(df.dim ==dim) & (df.nonlin_sem==nl)]
            print("----------------------------------")
            print("----------------------------------")
            print(f"{dim=} ({'linear' if nl is False else 'nonlinear'})\tMCC={mcc.mean():.3f}+{mcc.std():.3f}\tAcc:{np.array(success).mean()}\t SHD:{np.array(hamming).mean()}\t[{len(success)} items]")
            print("----------------------------------")
            print("----------------------------------")

## 3D SEM

In [128]:
learning_stats(df3d, true_unmix_jacobians3d, est_unmix_jacobians3d, permute_indices3d)

Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
----------------------------------
----------------------------------
dim=3 (linear)	MCC=1.000+0.000	Acc:1.0	 SHD:0.0	[24 items]
----------------------------------
----------------------------------
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
C

## 5D SEM

In [129]:
learning_stats(df5d, true_unmix_jacobians5d, est_unmix_jacobians5d, permute_indices5d, 1e-3)

Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
Correct order identified
----------------------------------
----------------------------------
dim=5 (linear)	MCC=1.000+0.000	Acc:1.0	 SHD:0.0	[7 items]
----------------------------------
----------------------------------
----------------------------------
----------------------------------
dim=5 (nonlinear)	MCC=nan+nan	Acc:nan	 SHD:nan	[0 items]
----------------------------------
----------------------------------


/var/folders/rb/d8k1n6bj4lg801y0yxz4jtbh0000gn/T/ipykernel_95175/2071231733.py:16: RuntimeWarning: Mean of empty slice.
  print(f"{dim=} ({'linear' if nl is False else 'nonlinear'})\tMCC={mcc.mean():.3f}+{mcc.std():.3f}\tAcc:{np.array(success).mean()}\t SHD:{np.array(hamming).mean()}\t[{len(success)} items]")
/Users/patrik.reizinger/Documents/GitHub/nl-causal-representations/venv/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## 8D SEM

## 10D SEM

## MLP from Monti et al.

In [77]:
learning_stats(df_monti, true_unmix_jacobians_monti, est_unmix_jacobians_monti, permute_indices_monti)

IndexError: tuple index out of range

In [83]:
success = []
for (mcc, j_gt, j_est, permute) in zip(df_monti.mcc, true_unmix_jacobians_monti, est_unmix_jacobians_monti, permute_indices_monti):
    if mcc > .95:
        success.append(learn_permutation(j_gt, j_est, permute, triu_weigth=20., tril_weight=10.,diag_weight=6., num_steps=5000, lr=1e-4, verbose=True, drop_smallest=True, threshold=None,binary=True))

print("----------------------------------")
print("----------------------------------")
print(f"Success ratio:{np.array(success).mean()}")
print("----------------------------------")
print("----------------------------------")

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.